(always be aware of your imports and <b><u><i>preserve namespaces</i></u></b>!!!)

In [1]:
import os
import sys
import time
import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt

plt.ion()
plt.rcParams["image.cmap"] = "gist_gray"

---

## Advanced (and relational) indexing: Part 1

Numpy array indexing is fast and allows you to manipulate images with only a few lines of code.  As an example, we'll create the "spotlight" widget.

First, it is often useful to create row and column index "maps":

In [2]:
dpath  = "images"
fname  = "city_image.jpg"
infile = os.path.join(dpath, fname)
img    = nd.imread(infile)

nrow, ncol = img.shape[:2]
rind = np.arange(nrow * ncol).reshape(nrow, ncol) // ncol
cind = np.arange(nrow * ncol).reshape(nrow, ncol) % ncol

In [3]:
fig8, ax8 = plt.subplots(1, 2, num=8, figsize=(8, 3))
ax8[0].imshow(rind)
ax8[0].grid(0)
ax8[1].imshow(cind)
ax8[1].grid(0)

which will let us index on positions in the image through "relational" indexing.  For example, we can create a mask for our image,

In [6]:
mask = np.dstack([(rind < 200).astype(np.uint8) for i in range(3)])

In [7]:
mask.shape

(448, 1040, 3)

In [8]:
ysize = 3.
xsize = ysize*float(img.shape[1]) / float(img.shape[0])

fig9, ax9 = plt.subplots(num=9, figsize=[xsize, ysize])
fig9.subplots_adjust(0, 0, 1, 1)
ax9.axis('off')
im9 = ax9.imshow(img * mask)
fig9.canvas.draw()

So, let's make a circular aperature around a point in the image:

In [9]:
rm, cm = 244, 302
dist = np.sqrt((rind - rm)**2 + (cind - cm)**2)

im9.set_data(dist)
im9.set_clim(0, 500)
fig9.canvas.draw()

In [10]:
dist.shape

(448, 1040)

In [12]:
mask = np.zeros(img.shape, dtype=np.uint8)

In [13]:
mask.shape

(448, 1040, 3)

In [15]:
(dist <= 100).shape

(448, 1040)

In [18]:
mask[dist <= 100] = [1, 1, 1, 4]

ValueError: shape mismatch: value array of shape (4,) could not be broadcast to indexing result of shape (31417,3)

In [19]:
mask[dist <= 100] = [1, 1, 1]

im9.set_data(255 * mask)
fig9.canvas.draw()

In [20]:
im9.set_data(img * mask)
fig9.canvas.draw()

---

## Interactive Matplotlib: Part 1

#### using Matplotlib's ginput() function

The ginput() function let's you click on a point and grab its location from the matplotlib window.  For example,

In [21]:
xpos, ypos = fig9.ginput()[0]

print("xpos = {0}\nypos = {1}".format(xpos, ypos))

/Users/gdobler/anaconda/lib/python2.7/site-packages/matplotlib/backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


xpos = 628.58045977
ypos = 195.109195402


In [22]:
fig9.ginput?

In [23]:
print fig9.ginput(3)

[(77.201149425287355, 78.557471264367734), (510.5344827586207, 181.66091954022983), (780.99425287356314, 72.580459770114885)]


In [24]:
cpos, rpos = [int(round(i)) for i in fig9.ginput()[0]]

print("rpos = {0}\ncpos = {1}".format(rpos, cpos))

rpos = 241
cpos = 612


### We now have all the tools we need to make the spotlight widget!!!

1. read in an image with scipy.ndimage
2. display it using matplotlib
3. mask the whole image so it is all black<br>
    a. create row and column index grid<br>
    b. create a mask with all zeros
4. grab a point off of the image using ginput
5. set the mask pixels within some aperature to 1
6. redisplay the masked image
7. put steps 4-6 in a loop

---

In [16]:
plt.close("all")